In [3]:
import numpy as np
import pandas as pd
import json
from scipy.signal import find_peaks
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
import squidpy as sq

import src.utilities as utils
from src.slide_utilities import *

import warnings
warnings.filterwarnings('ignore')

In [6]:
adata_main = sc.read_h5ad("/data/projects/robin/segmentation/xenium_outs/merged_processed_cleaned.h5ad")
del adata_main.layers['counts']

Add domain generated by ```01_nichepca_per_sample.ipynb``` to ```adata_main```

In [7]:
adata_main.obs['sample'] = [i.split('_')[0] for i in adata_main.obs['Patient_Sample_ID']]
sample_set = adata_main.obs['sample'].unique()
sample_set

array(['X40', 'X39', 'X38', 'X37', 'X36', 'X35', 'X34', 'X33', 'X63',
       'X61', 'X62', 'X60', 'X59', 'X58', 'X57', 'X32', 'X31', 'X30',
       'X29', 'X28', 'X27', 'X26', 'X25', 'X56', 'X55', 'X54', 'X53',
       'X52', 'X50', 'X51', 'X49', 'X8', 'X7', 'X6', 'X4', 'X5', 'X2',
       'X3', 'X1', 'X48', 'X47', 'X46', 'X45', 'X44', 'X43', 'X42', 'X41',
       'X16', 'X15', 'X14', 'X12', 'X13', 'X11', 'X10', 'X9', 'X24',
       'X23', 'X22', 'X21', 'X20', 'X19', 'X18', 'X17'], dtype=object)

In [21]:
resolutions = np.arange(0.03, 0.4, 0.02)
resolutions = resolutions.round(4).tolist()
for res in resolutions:
    adata_main.obs[f'per_sample_leiden_res_{res}'] = '0'

In [22]:
data_path = "/projectbig/jupyternotebook/xenium_cgn/domain_identification_from_epyc/per_sample_knn/"

for sample in tqdm(sample_set):
    adata_per_sample = sc.read_h5ad(data_path + f"adata_nichepca_sample_{sample}_clustered.h5ad")
    
    for res in resolutions:
        adata_main.obs.loc[adata_main.obs['sample'] == sample ,f'per_sample_leiden_res_{res}'] = adata_per_sample.obs[f'leiden_res_{res}']

    # if sample in ['X60', 'X25', 'X7', 'X44', 'X10']:
    #     adata_main.write("data/adata_nichepca_slide_clustered_per_sample_tmp.h5ad")

100%|██████████| 63/63 [1:35:33<00:00, 91.02s/it] 


In [23]:
adata_main.obs

,x,y,z,cluster,n_transcripts,density,elongation,area,avg_confidence,avg_assignment_confidence,...,per_sample_leiden_res_0.21,per_sample_leiden_res_0.23,per_sample_leiden_res_0.25,per_sample_leiden_res_0.27,per_sample_leiden_res_0.29,per_sample_leiden_res_0.31,per_sample_leiden_res_0.33,per_sample_leiden_res_0.35,per_sample_leiden_res_0.37,per_sample_leiden_res_0.39
cell_CRfb095a77d-1,2285.790178,1164.057381,32.829188,1,79,2.0240,2.141,39.03,0.9975,0.9916,...,1,1,2,2,2,2,2,2,2,2
cell_CRfb095a77d-2,2298.342622,1219.741069,31.465535,4,32,1.0850,4.614,29.50,0.9771,0.9288,...,1,1,2,2,2,2,2,2,2,2
cell_CRfb095a77d-3,2271.434052,1245.859500,32.062796,2,67,3.0590,2.990,21.90,0.9971,0.9884,...,3,3,4,4,4,4,4,5,4,7
cell_CRfb095a77d-5,2257.346907,1259.955827,31.273945,2,56,2.1420,9.229,26.14,0.9964,0.9707,...,3,3,4,4,4,4,4,5,4,7
cell_CRfb095a77d-6,2277.712022,1237.595244,35.395148,2,9,1.6980,3.283,5.30,0.9826,0.9400,...,3,3,4,4,4,4,4,5,4,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cell_CRc71d71060-597925,2747.360264,4957.516627,24.097605,2,73,1.5480,3.617,47.15,0.9971,0.6334,...,1,1,1,1,1,1,1,1,1,1
cell_CRc71d71060-598041,4592.548620,1655.438877,24.254944,2,35,0.9893,1.844,35.38,0.9951,0.6206,...,1,1,1,1,1,1,1,1,1,1
cell_CRc71d71060-598270,8540.113826,9224.561804,19.515240,3,46,1.3870,2.667,33.16,0.9969,0.7152,...,1,1,1,1,1,1,1,1,1,1
cell_CRc71d71060-598508,4264.902000,2582.697217,25.121288,3,24,1.1420,5.004,21.02,0.9914,0.7517,...,1,1,6,6,6,6,6,6,7,7


In [24]:
adata_main.write("data/adata_nichepca_slide_clustered_per_sample_tmp.h5ad")
"data/adata_nichepca_slide_clustered_per_sample_tmp.h5ad"

'data/adata_nichepca_slide_clustered_per_sample_tmp.h5ad'